In [1]:
import numpy as np
import pandas as pd
import sys

#data_dir = '/content/drive/MyDrive/????'
#sys.path.insert(0, data_dir)

In [2]:
sb = pd.read_csv('./스타벅스.csv')
cb = pd.read_csv('./커피빈.csv')
ed = pd.read_csv('./이디야.csv')
bd = pd.read_csv('./빽다방.csv')
sb

,상호명,시도명,시군구명
0,스타벅스강릉안목항점,강원도,강릉시
1,스타벅스춘천후평DT점,강원도,춘천시
2,스타벅스,강원도,춘천시
3,스타벅스설악워터피아점,강원도,속초시
4,스타벅스,강원도,원주시
...,...,...,...
1499,스타벅스,충청북도,청주시 서원구
1500,스타벅스,충청북도,청주시 청원구
1501,스타벅스,충청북도,충주시
1502,스타벅스,충청북도,음성군


In [3]:
# 시군구명이 겹치는 시가 있음
# ex) 고성군 - 강원도, 경상남도
#     광역시 - 서울 용산, 서울 서대문, 대전 서구, 대전 유성, 세종
#     행정구 - 수원 장안, 요인 수지, 고양 일산동, 창원 합포, 창원 회원

# 처리
tmp_gu_dict = {'수원': ['장안구', '권선구', '팔달구', '영통구'],
               '성남': ['수정구', '중원구', '분당구'],
               '안양': ['만안구', '동안구'],
               '안산': ['상록구', '단원구'],
               '고양': ['덕양구', '일산동구', '일산서구'],
               '용인': ['처인구', '기흥구', '수지구'],
               '청주': ['상당구', '서원구', '흥덕구'],
               '천안': ['동남구', '서북구'],
               '전주': ['완산구', '덕진구'],
               '포항': ['남구', '북구'],
               '창원': ['의창구', '성산구', '진해구', '마산합포구', '마산회원구']
               }

In [4]:
metro_list = [ '서울특별시', '부산광역시', '대구광역시', '인천광역시', '대전광역시', '광주광역시', '울산광역시' ]

In [5]:
def get_id(df):

    # 시네임 체크
    si_name = [None] * len(df)  # 없으면 0, 리스트 반환

    for i in df.index:
        if df.시도명[i] in metro_list:
            if len(df.시군구명[i]) == 2:
                # 서울 중구, 대전 서구
                si_name[i] = df.시도명[i][:2] + ' ' + df.시군구명[i]
            else:
                # 서울 용산, 서울 서대문
                si_name[i] = df.시도명[i][:2] + ' ' + df.시군구명[i][:-1]
        else:
            si_len = len(df.시군구명[i].split())
            if si_len == 1:
                if df.시군구명[i][:-1] == '고성':
                    if df.시도명[i] == '강원도':
                        si_name[i] = '고성(강원도)'
                    else:
                        si_name[i] = '고성(경남)'
                elif df.시군구명[i].find('세종') == 0:
                    si_name[i] = '세종'
                else:
                    # 세종, 광명, 김포, 강릉
                    si_name[i] = df.시군구명[i][:-1]
            else:
                admin_gu = df.시군구명[i].split()[-1]
                for key, value in tmp_gu_dict.items():
                    if admin_gu in value:
                        if len(admin_gu) == 2:
                            # 포항 북구, 포항 남구
                            si_name[i] = key + ' ' + admin_gu
                        elif len(admin_gu) == 5:
                            # 창원 합포, 창원 회원
                            si_name[i] = key + ' ' + admin_gu[2:-1]
                        else:
                            # 수원 팔달, 용인 지
                            si_name[i] = key + ' ' + admin_gu[:-1]
    return si_name


In [6]:
sb.head()

,상호명,시도명,시군구명
0,스타벅스강릉안목항점,강원도,강릉시
1,스타벅스춘천후평DT점,강원도,춘천시
2,스타벅스,강원도,춘천시
3,스타벅스설악워터피아점,강원도,속초시
4,스타벅스,강원도,원주시


In [22]:
sb['ID'] = get_id(sb)
cb['ID'] = get_id(cb)
ed['ID'] = get_id(ed)
bd['ID'] = get_id(bd)
print('스벅: ', len(sb.ID.unique()))
print('커빈: ', len(cb.ID.unique()))
print('이디: ', len(ed.ID.unique()))
print('빽다: ', len(bd.ID.unique()))
sb

스벅:  191
커빈:  92
이디:  238
빽다:  174


,상호명,시도명,시군구명,ID
0,스타벅스강릉안목항점,강원도,강릉시,강릉
1,스타벅스춘천후평DT점,강원도,춘천시,춘천
2,스타벅스,강원도,춘천시,춘천
3,스타벅스설악워터피아점,강원도,속초시,속초
4,스타벅스,강원도,원주시,원주
...,...,...,...,...
1499,스타벅스,충청북도,청주시 서원구,청주 서원
1500,스타벅스,충청북도,청주시 청원구,None
1501,스타벅스,충청북도,충주시,충주
1502,스타벅스,충청북도,음성군,음성


In [23]:
# 모든 단위 도시를 합하면 합집합
city_set = set(sb.ID.unique()) | set(cb.ID.unique()) | set(ed.ID.unique()) | set(bd.ID.unique())
len(set(city_set))

244

In [9]:
# 스타벅스만 있고, 이디야 / 빽다방이 없는 곳
set(sb.ID.unique()) - set(ed.ID.unique()) - set(bd.ID.unique())

{'계룡', '고성(강원도)', '영양', '진도'}

In [10]:
# 브랜드별 매장수 계산
# 스벅
sb.groupby(by='ID')[['상호명']].count().head()

,상호명
ID,
가평,2
강릉,4
거제,6
경산,8
경주,7


In [21]:
pt_sb = sb.groupby('ID')[['상호명']].count()
pt_sb.reset_index(inplace=True)
pt_sb.rename(columns={'상호명':'스타벅스'}, inplace=True)

pt_cb = cb.groupby('ID')[['상호명']].count()
pt_cb.reset_index(inplace=True)
pt_cb.rename(columns={'상호명':'커피빈'}, inplace=True)

pt_ed = ed.groupby('ID')[['상호명']].count()
pt_ed.reset_index(inplace=True)
pt_ed.rename(columns={'상호명':'이디야'}, inplace=True)

pt_bd = bd.groupby('ID')[['상호명']].count()
pt_bd.reset_index(inplace=True)
pt_bd.rename(columns={'상호명':'빽다방'}, inplace=True)

len(pt_sb), len(pt_cb), len(pt_ed), len(pt_bd)
pt_sb

,ID,스타벅스
0,가평,2
1,강릉,4
2,거제,6
3,경산,8
4,경주,7
...,...,...
185,하남,10
186,하동,1
187,홍천,2
188,화성,19


In [12]:
# merge로 커피데이터 병합, 이디야랑 스벅만
cf = pd.merge(pt_ed, pt_sb, how='left')
cf.head()

,ID,이디야,스타벅스
0,가평,9,2.0
1,강릉,17,4.0
2,강진,1,NaN
3,거제,9,6.0
4,거창,1,NaN


In [13]:
cf = pd.merge(cf, pt_cb, how='left')
cf = pd.merge(cf, pt_bd, how='left')
cf.head()

,ID,이디야,스타벅스,커피빈,빽다방
0,가평,9,2.0,NaN,NaN
1,강릉,17,4.0,NaN,2.0
2,강진,1,NaN,NaN,NaN
3,거제,9,6.0,NaN,NaN
4,거창,1,NaN,NaN,1.0


In [16]:
# 결측 0으로 대체
cf.fillna(0, inplace=True)
cf.스타벅스 = cf.스타벅스.astype(int)
cf.이디야 = cf.이디야.astype(int)
cf.커피빈 = cf.커피빈.astype(int)
cf.빽다방 = cf.빽다방.astype(int)
cf.head()

,ID,이디야,스타벅스,커피빈,빽다방
0,가평,9,2,0,0
1,강릉,17,4,0,2
2,강진,1,0,0,0
3,거제,9,6,0,0
4,거창,1,0,0,1


In [20]:
cf['커피지수'] = (cf.스타벅스 + cf.커피빈) / (cf.이디야+cf.빽다방)
cf.to_csv('/content/drive/MyDrive/2023 직업훈련_AI/실습데이터/커피지수.csv')